In [84]:
import pandas as pd
import numpy as np
import plotly.express as px

In [85]:
df_crops = pd.read_csv("../data/all_crop_data.csv")
df_corn = df_crops[(df_crops["Country_Name"] == "United States") & (df_crops["Commodity_Description"] == "Corn")].reset_index(drop=True)

We only care about Soybeans, Wheat, Cotton, and Corn. Let's make sure we're only considering those crops before doring any analysis.

In [86]:
df_crops.head()

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,20,Beginning Stocks,21,(MT),0.0
1,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,125,Domestic Consumption,21,(MT),0.0
2,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,176,Ending Stocks,21,(MT),0.0
3,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,88,Exports,21,(MT),0.0
4,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,57,Imports,21,(MT),0.0


In [87]:
df_crops["Commodity_Description"].unique()

array(['Almonds, Shelled Basis', 'Animal Numbers, Cattle',
       'Animal Numbers, Swine', 'Apples, Fresh', 'Barley',
       'Cherries (Sweet&Sour), Fresh', 'Coffee, Green', 'Corn', 'Cotton',
       'Dairy, Butter', 'Dairy, Cheese', 'Dairy, Dry Whole Milk Powder',
       'Dairy, Milk, Fluid', 'Dairy, Milk, Nonfat Dry',
       'Grapefruit, Fresh', 'Grapes, Fresh Table', 'Lemons/Limes, Fresh',
       'Meal, Copra', 'Meal, Cottonseed', 'Meal, Fish',
       'Meal, Palm Kernel', 'Meal, Peanut', 'Meal, Rapeseed',
       'Meal, Soybean', 'Meal, Soybean (Local)', 'Meal, Sunflowerseed',
       'Meat, Beef and Veal', 'Meat, Chicken', 'Meat, Swine', 'Millet',
       'Mixed Grain', 'Oats', 'Oil, Coconut', 'Oil, Cottonseed',
       'Oil, Olive', 'Oil, Palm', 'Oil, Palm Kernel', 'Oil, Peanut',
       'Oil, Rapeseed', 'Oil, Soybean', 'Oil, Soybean (Local)',
       'Oil, Sunflowerseed', 'Oilseed, Copra', 'Oilseed, Cottonseed',
       'Oilseed, Palm Kernel', 'Oilseed, Peanut', 'Oilseed, Rapeseed',
    

In [88]:
# Ensure only the relevent crops remain in the dataframe
crops = ["Soybean", "Wheat", "Cotton", "Corn"]
df_crops = df_crops[df_crops["Commodity_Description"].str.contains("|".join(crops))]

**Exploratory Data Analysis**

In [89]:
df_crops["Commodity_Description"].unique()

array(['Corn', 'Cotton', 'Meal, Cottonseed', 'Meal, Soybean',
       'Meal, Soybean (Local)', 'Oil, Cottonseed', 'Oil, Soybean',
       'Oil, Soybean (Local)', 'Oilseed, Cottonseed', 'Oilseed, Soybean',
       'Oilseed, Soybean (Local)', 'Wheat'], dtype=object)

In [90]:
df_crops = df_crops[df_crops["Attribute_Description"] == ("Production")]
df_crops["Commodity_Description"].unique()

array(['Corn', 'Cotton', 'Meal, Cottonseed', 'Meal, Soybean',
       'Meal, Soybean (Local)', 'Oil, Cottonseed', 'Oil, Soybean',
       'Oil, Soybean (Local)', 'Oilseed, Cottonseed', 'Oilseed, Soybean',
       'Oilseed, Soybean (Local)', 'Wheat'], dtype=object)

According to the PSD USDA frequently asked questions page, "Oil, " and "Meal," denotations are simply derivatives of Oilseed products. This explains why those derivatives do not have a Yield value associated with them - i.e. Oilseeds are the harvested crops while Oil and Meal are simply byproducts of the harvest.

Thus, let's continue by removing all Oil, Soybean/Cottonseed and Meal, Soybean/Cottonseed rows and aggregating the remaining (i.e. there is no distinction with (Local) in our use case.) rows.

In [91]:
labels_to_parse = [label for label in df_crops["Commodity_Description"].unique() if "Local" in label]
for label in labels_to_parse:
    typeOfCrop, name, _ = label.split(" ")
    df_crops.loc[df_crops["Commodity_Description"].str.contains(f"{name} (Local)", regex=False), "Commodity_Description"] = f"{typeOfCrop} {name}"

In [92]:
cropLabels = df_crops["Commodity_Description"].unique()
cropLabels

array(['Corn', 'Cotton', 'Meal, Cottonseed', 'Meal, Soybean',
       'Oil, Cottonseed', 'Oil, Soybean', 'Oilseed, Cottonseed',
       'Oilseed, Soybean', 'Wheat'], dtype=object)

In [93]:
# # Convert cotton units into Metric Tons/Hectare to make it consistent with the other crop values
# df_crops.loc[df_crops["Commodity_Description"] == "Cotton", "Value"] /= 1000
# df_crops.loc[df_crops["Commodity_Description"] == "Cotton", "Unit_Description"] = "(MT/HA)"
# df_crops.loc[df_crops["Commodity_Description"] == "Cotton", "Unit_ID"] = "26"


# # Remove rows where the yield is 0
df_crops = df_crops[df_crops["Value"] != 0]


In [94]:
totalYieldByCountry = df_crops[(df_crops["Attribute_Description"].str.contains("Production"))]\
                      .groupby(by=["Commodity_Description", "Country_Name", "Unit_Description"])\
                      .agg({"Value": "mean"}).sort_values(by="Value", ascending=False)

In [95]:
totalYieldByCountry["Value"]["Cotton"].head(10)

Country_Name                    Unit_Description  
China                           1000 480 lb. Bales    19611.904762
United States                   1000 480 lb. Bales    14960.174603
India                           1000 480 lb. Bales    13202.571429
Union of Soviet Socialist Repu  1000 480 lb. Bales    10354.444444
Pakistan                        1000 480 lb. Bales     5726.079365
Uzbekistan                      1000 480 lb. Bales     4854.305556
Brazil                          1000 480 lb. Bales     4503.507937
Turkey                          1000 480 lb. Bales     2714.746032
Australia                       1000 480 lb. Bales     1685.111111
Egypt                           1000 480 lb. Bales     1423.714286
Name: Value, dtype: float64

In [96]:
totalYieldByCountry["Value"]["Wheat"].head(10)

Country_Name                    Unit_Description
European Union                  (1000 MT)           135132.958333
China                           (1000 MT)            82463.968254
Union of Soviet Socialist Repu  (1000 MT)            77908.740741
EU-15                           (1000 MT)            65630.333333
India                           (1000 MT)            54108.603175
United States                   (1000 MT)            54059.555556
Russia                          (1000 MT)            51095.500000
Canada                          (1000 MT)            23414.349206
Ukraine                         (1000 MT)            20415.333333
Australia                       (1000 MT)            16925.015873
Name: Value, dtype: float64

In [97]:
totalYieldByCountry["Value"]["Oilseed, Soybean"].head(10)

Country_Name    Unit_Description
United States   (1000 MT)           65715.932203
Brazil          (1000 MT)           52486.978261
Argentina       (1000 MT)           27314.782609
China           (1000 MT)           12049.152542
India           (1000 MT)            4702.148148
Paraguay        (1000 MT)            3191.542373
Canada          (1000 MT)            2420.559322
European Union  (1000 MT)            1657.208333
Russia          (1000 MT)            1455.888889
Ukraine         (1000 MT)            1406.888889
Name: Value, dtype: float64

In [98]:
totalYieldByCountry["Value"]["Oilseed, Cottonseed"].head(10)

Country_Name                    Unit_Description
China                           (1000 MT)           8330.037037
India                           (1000 MT)           6260.074074
United States                   (1000 MT)           5019.037037
Union of Soviet Socialist Repu  (1000 MT)           4491.000000
Pakistan                        (1000 MT)           2769.537037
Uzbekistan                      (1000 MT)           1982.500000
Brazil                          (1000 MT)           1699.111111
Turkey                          (1000 MT)            977.703704
EU-15                           (1000 MT)            627.500000
Australia                       (1000 MT)            605.370370
Name: Value, dtype: float64

In [99]:
totalYieldByCountry["Value"]["Corn"].head(10)

Country_Name                    Unit_Description
United States                   (1000 MT)           220282.507937
China                           (1000 MT)           112578.380952
European Union                  (1000 MT)            61550.250000
Brazil                          (1000 MT)            38665.285714
EU-15                           (1000 MT)            20957.641026
Argentina                       (1000 MT)            16485.682540
Mexico                          (1000 MT)            15447.460317
Ukraine                         (1000 MT)            13684.972222
India                           (1000 MT)            12396.206349
Union of Soviet Socialist Repu  (1000 MT)            11114.703704
Name: Value, dtype: float64

In [100]:
# Let's see the date range of the data we have available to us

print("Earliest Record:", df_crops["Market_Year"].min(), "\nLatest Record:", df_crops["Market_Year"].max())
print("Number of Records:", len(df_crops))

# Frequency by crop
df_crops["Commodity_Description"].value_counts()

Earliest Record: 1960 
Latest Record: 2022
Number of Records: 31461


Corn                   6576
Cotton                 4863
Wheat                  4699
Meal, Soybean          3546
Oil, Soybean           2951
Oilseed, Soybean       2394
Meal, Cottonseed       2217
Oil, Cottonseed        2180
Oilseed, Cottonseed    2035
Name: Commodity_Description, dtype: int64

Let's analyze specific countries/regions based off their total productions to see if we can refine the scope of our analysis.

**United States Analysis**

In [101]:
df_corn_USA = df_crops[(df_crops["Country_Name"] == "United States") & (df_crops["Commodity_Description"] == "Corn")].reset_index(drop=True)

In [102]:
df_corn_USA.head()

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,440000,Corn,US,United States,1960,2006,7,28,Production,8,(1000 MT),99241.0
1,440000,Corn,US,United States,1961,2006,7,28,Production,8,(1000 MT),91389.0
2,440000,Corn,US,United States,1962,2006,7,28,Production,8,(1000 MT),91605.0
3,440000,Corn,US,United States,1963,2006,7,28,Production,8,(1000 MT),102093.0
4,440000,Corn,US,United States,1964,2006,7,28,Production,8,(1000 MT),88504.0


In [103]:
px.bar(df_corn_USA, x="Market_Year", y="Value", title="United States Corn Yield over Time")

**Australia Region Analysis**

In [104]:
df_crops_Australia = df_crops[(df_crops["Country_Name"].str.contains("Australia"))]
px.bar(df_crops_Australia, x="Market_Year", y="Value", title="Australia Crop Yield over Time", color="Commodity_Description")

Let's settle on exploring the United State's production of **corn** for the rest of this exploration. (Since they are the world's top producer of Corn afterall)

In [105]:
df_corn_USA.tail()

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
58,440000,Corn,US,United States,2018,2021,7,28,Production,8,(1000 MT),364262.0
59,440000,Corn,US,United States,2019,2022,8,28,Production,8,(1000 MT),345962.0
60,440000,Corn,US,United States,2020,2022,11,28,Production,8,(1000 MT),358447.0
61,440000,Corn,US,United States,2021,2022,11,28,Production,8,(1000 MT),382893.0
62,440000,Corn,US,United States,2022,2022,11,28,Production,8,(1000 MT),353836.0


In [106]:
# df_corn_USA["Bushels"] = df_corn_USA["Production"]*39.368
# df_corn_USA["Bushels"]

In [135]:
# Taken from https://ipad.fas.usda.gov/countrysummary/Default.aspx?id=US&crop=Corn
corn_belt_states = {
    "Minnesota": .1,
    "South Dakota": .06,
    "Nebraska": .12,
    "Kansas": .06,
    "Iowa": .17,
    "Wisconsin": .04,
    "Illinois": .15,
    "Missouri": .04,
    "Indiana": .07,
    "Ohio": .04
}
sum(corn_belt_states.values())

0.8500000000000001

Let's simplify the regions we're analyzing to be that of the corn belt states since they 
account for ~85% of the USA's corn production.

In [108]:
df_corn_USA.iloc[-10:]

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
53,440000,Corn,US,United States,2013,2019,2,28,Production,8,(1000 MT),351316.0
54,440000,Corn,US,United States,2014,2019,2,28,Production,8,(1000 MT),361136.0
55,440000,Corn,US,United States,2015,2019,7,28,Production,8,(1000 MT),345506.0
56,440000,Corn,US,United States,2016,2020,7,28,Production,8,(1000 MT),384778.0
57,440000,Corn,US,United States,2017,2021,7,28,Production,8,(1000 MT),371096.0
58,440000,Corn,US,United States,2018,2021,7,28,Production,8,(1000 MT),364262.0
59,440000,Corn,US,United States,2019,2022,8,28,Production,8,(1000 MT),345962.0
60,440000,Corn,US,United States,2020,2022,11,28,Production,8,(1000 MT),358447.0
61,440000,Corn,US,United States,2021,2022,11,28,Production,8,(1000 MT),382893.0
62,440000,Corn,US,United States,2022,2022,11,28,Production,8,(1000 MT),353836.0


We really only care about the year and **production** value columns at this point. Furthermore, let's analyze data all the way up to the last decade
since more data implies better results. 

Plus, any earlier data might be inconsistent with our current distribution since we're not considering
any changes in agricultural technology/practices that could potentially affect production

In [117]:
df_final_crops = df_corn_USA[["Market_Year", "Value"]].rename(columns={"Value": "Total_Production"}).iloc[-10:].reset_index(drop=True)
df_final_crops

,Market_Year,Total_Production
0,2013,351316.0
1,2014,361136.0
2,2015,345506.0
3,2016,384778.0
4,2017,371096.0
5,2018,364262.0
6,2019,345962.0
7,2020,358447.0
8,2021,382893.0
9,2022,353836.0


Now, let's partition these production values by state (according to our mapping).

**Note**: This generalization can be done only because all of the states of the corn belt tend to have
similar climatic factors (i.e. soil condition, terrain, sunlight exposure, etc.).

In [134]:
for state in corn_belt_states:
    df_final_crops[f"{state.lower()}_production"] = df_final_crops["Total_Production"] * corn_belt_states[state]


In [152]:
# Validate that the sum of the corn belt's production is strictly 85% of the total production for that year
for _, row in df_final_crops.iterrows():

    expectedProduction = row[1] * 0.85
    cornBeltProduction = sum(row[2:])
    print(expectedProduction - cornBeltProduction)

0.0
0.0
0.0
0.0
-5.820766091346741e-11
0.0
0.0
0.0
-5.820766091346741e-11
0.0


In [154]:
# Now that we've validated our processing, we can drop the total_production column since it's not needed anymore
df_final_crops = df_final_crops.rename(columns={"Market_Year": "year"})
df_final_crops = df_final_crops.drop(columns=["Total_Production"])
df_final_crops

,year,minnesota_production,south dakota_production,nebraska_production,kansas_production,iowa_production,wisconsin_production,illinois_production,missouri_production,indiana_production,ohio_production
0,2013,35131.6,21078.96,42157.92,21078.96,59723.72,14052.64,52697.40,14052.64,24592.12,14052.64
1,2014,36113.6,21668.16,43336.32,21668.16,61393.12,14445.44,54170.40,14445.44,25279.52,14445.44
2,2015,34550.6,20730.36,41460.72,20730.36,58736.02,13820.24,51825.90,13820.24,24185.42,13820.24
3,2016,38477.8,23086.68,46173.36,23086.68,65412.26,15391.12,57716.70,15391.12,26934.46,15391.12
4,2017,37109.6,22265.76,44531.52,22265.76,63086.32,14843.84,55664.40,14843.84,25976.72,14843.84
5,2018,36426.2,21855.72,43711.44,21855.72,61924.54,14570.48,54639.30,14570.48,25498.34,14570.48
6,2019,34596.2,20757.72,41515.44,20757.72,58813.54,13838.48,51894.30,13838.48,24217.34,13838.48
7,2020,35844.7,21506.82,43013.64,21506.82,60935.99,14337.88,53767.05,14337.88,25091.29,14337.88
8,2021,38289.3,22973.58,45947.16,22973.58,65091.81,15315.72,57433.95,15315.72,26802.51,15315.72
9,2022,35383.6,21230.16,42460.32,21230.16,60152.12,14153.44,53075.40,14153.44,24768.52,14153.44


In [158]:
df_final_crops.to_csv("../data/crop_data.csv", index=False)

Let's actually pivot because we most likely will not have enough data for modeling.

Instead, let's pull production samples on the county-level for each of these corn belt states.

Further EDA/data set creation work will happen in "even_more_crops_analysis.ipynb"